In [1]:
# Import packages

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import boto3

import _pickle as cpickle 

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/mymodule/')

from common_functions import psi
from common_functions import ks
from common_functions import data_vars
%matplotlib inline


In [2]:
def download_model(model_name):
    #model_name = 'slowsecure_xgb_model_w'
    s3.meta.client.download_file(bucket_name, model_path + model_name, model_name)
    with open(model_name, 'rb') as f:
        xgb = cpickle.load(f)
        return xgb

In [3]:
df_columns = pd.read_csv('pps_attributes.csv')


In [4]:
slowsecure_columns = df_columns[ df_columns['segment'] == 'slowsecure']['columns_name'].to_list()
slowunsecure_columns = df_columns[ df_columns['segment'] == 'slowunsecure']['columns_name'].to_list()
slownoappl_columns = df_columns[ df_columns['segment'] == 'slownoappl']['columns_name'].to_list()
pot30_columns = df_columns[ df_columns['segment'] == 'pot30']['columns_name'].to_list()
pot60_columns = df_columns[ df_columns['segment'] == 'pot60']['columns_name'].to_list()
pot90_columns = df_columns[ df_columns['segment'] == 'pot90']['columns_name'].to_list()

# char analysis for slowsecure segment

In [5]:
#read in the csv model development data

# df1=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202008/secure.csv')
df2=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/secure.csv')
df3=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202010/secure.csv')
df_dev=pd.concat([df2, df3],ignore_index=True)
df_dev.shape

(124201, 618)

In [8]:

#old s3_source_path='s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/'
YYYYMM = '202103'
bucket_name = 'mf-ml-ds'
s3_source = 's3://'+ bucket_name +'/propensity_to_pay/Monthly_Validation/val_'
s3_source_path  = s3_source + YYYYMM
s3_score_target  = 'propensity_to_pay/Monthly_Validation/val_' + YYYYMM +'/'
# pps_score_file_name = 'xgb_weighted_score_pot_' + YYYYMM +'.csv'

psi_file_path = 's3://'+ bucket_name + '/' + s3_score_target
model_path = 'propensity_to_pay/final_model/'

s3 = boto3.resource('s3')


In [9]:
df_val=pd.read_csv(s3_source_path+'/secure.csv')


In [10]:
df_dev=df_dev[slowsecure_columns]

df_val=df_val[slowsecure_columns]




In [11]:
df_dev.fillna(0,inplace=True)
df_val.fillna(0,inplace=True)

X_dev=df_dev.drop(['badflag'],axis=1)
X_val=df_val.drop(['badflag'],axis=1)

In [12]:
from scipy import stats

def ks_two_sample_test(X1,X2):
#     kolmogorov_smirnov=[[(0,0)]]*len(X1.columns)
    kolmogorov_smirnov= pd.DataFrame()
    # print kolmogorov_smirnov
    for feature_col in X1.columns:
            ks_test_statistic,p_value=stats.ks_2samp(X1[feature_col], X2[feature_col])
            kolmogorov_smirnov[feature_col]=(ks_test_statistic,p_value)

    return kolmogorov_smirnov 


In [13]:
ks_two_sample_test(X_dev, X_val)


,MOBVintageUnthread,__of_days_until_pmt_date,Times29_6mo,Times29_12mo,Times29_Ever,Times60_Ever,Times90_Ever,ADA_3368,ADA_3863,ADA_3842,...,ADA_3366,ADA_3612,ADA_3852,ADA_3856,ADA_3901,ADA_3144,ADA_3422,Fixed_Income,Hourly_Income,Auto_Pay_flag
0,8.037099e-02,2.157985e-02,4.583619e-02,2.438292e-02,2.541491e-02,0.004990,0.005173,1.691983e-02,0.006905,0.007345,...,2.499086e-02,2.306719e-02,0.009529,1.579808e-02,1.583499e-02,2.387768e-02,0.010905,0.003011,0.001808,4.419854e-02
1,8.607305e-233,3.538813e-17,5.293326e-76,8.207995e-22,1.162949e-23,0.253723,0.217603,1.006272e-10,0.038541,0.022915,...,6.830582e-23,1.445667e-19,0.001082,2.103574e-09,1.909677e-09,6.185395e-21,0.000106,0.847012,0.999250,1.066006e-70


In [14]:
def drift_feature(train, test, thresh=0.70):

## reading files
# train = X_dev
# test = X_val

## creating a new feature origin
    train['origin'] = 0
    test['origin'] = 1


## taking sample from training and test data
    if len(train)>10000:
        training = train.sample(10000, random_state=12)
    else: 
        training=train
    if len(test)>10000:
        testing = test.sample(10000, random_state=11)
    else:
        testing=test
        
## combining random samples
    combi = training.append(testing)
    y = combi['origin']
    combi.drop('origin',axis=1,inplace=True)

## modelling
    model = XGBClassifier(n_estimators=50, min_samples_leaf=5, randomstate=42)
    drop_list = []
    for i in combi.columns:
        score = cross_val_score(model,pd.DataFrame(combi[i]),y,cv=2,scoring='roc_auc')
 
        if (np.mean(score) > thresh):
            drop_list.append(i)
            print(i,np.mean(score))
    if len(drop_list)>0:
        print( "These are drifting features !")
    else:
        print( "These are no drifting features !")
    return drop_list


In [15]:
drop_list=drift_feature(X_dev, X_val, thresh=0.7)

These are no drifting features !


In [16]:
dev_final_iv, dev_IV = data_vars(df_dev, df_dev.badflag)
val_final_iv, val_IV = data_vars(df_val, df_val.badflag)
comp_iv=pd.merge(dev_IV, val_IV, on='VAR_NAME')
comp_iv = comp_iv.rename(columns={'IV_x': 'IV_dev', 'IV_y': 'IV_val'})
pd.options.display.float_format = '{:,.5f}'.format
comp_iv.IV_dev = comp_iv.IV_dev.round(4)
comp_iv.IV_val = comp_iv.IV_val.round(4)
comp_iv['Delta']=comp_iv['IV_val']-comp_iv['IV_dev']
comp_iv.sort_values(by='Delta', ascending=True)

,VAR_NAME,IV_dev,IV_val,Delta
20,ADA_3755,0.06000,0.01300,-0.04700
22,ADA_3761,0.12740,0.09680,-0.03060
7,ADA_3347,0.20080,0.17390,-0.02690
23,ADA_3842,0.15790,0.13410,-0.02380
4,ADA_3208,0.20080,0.17760,-0.02320
24,ADA_3852,0.07690,0.05770,-0.01920
28,ADA_3901,0.20860,0.19030,-0.01830
1,ADA_3026,0.01850,0.00490,-0.01360
25,ADA_3854,0.01980,0.00970,-0.01010
21,ADA_3757,0.02240,0.01620,-0.00620


# char analysis for slowunsecure segment

In [123]:
#read in the csv model development data


df_dev=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/unsecure.csv')

df_val=pd.read_csv(s3_source_path+'/unsecure.csv')

In [124]:
df_dev=df_dev[slowunsecure_columns]

df_val=df_val[slowunsecure_columns]


In [75]:
df_dev.fillna(0,inplace=True)
df_val.fillna(0,inplace=True)

X_dev=df_dev.drop(['badflag'],axis=1)
X_val=df_val.drop(['badflag'],axis=1)

In [76]:
ks_two_sample_test(X_dev, X_val)


,MOBVintageUnthread,__of_days_until_pmt_date,Times29_6mo,Times30_9mo,Times29_12mo,Times29_Ever,Times30_Ever,Times60_Ever,Times90_Ever,ADA_3217,...,ADA_3208,ADA_3386,ADA_3915,ADA_3979,ADA_3992,ADA_3422,Fixed_Income,Auto_Pay_flag,CreditKumar,upb_ratio
0,0.05644,0.01874,0.00299,0.00354,0.00735,0.00642,0.00445,0.00292,0.00226,0.00085,...,0.00508,0.00719,0.01786,0.01897,0.02309,0.00369,0.00521,0.03525,0.00712,0.03096
1,0.00000,0.00000,0.11386,0.03563,0.00000,0.00000,0.00339,0.12963,0.38444,0.99984,...,0.00049,0.00000,0.00000,0.00000,0.00000,0.02511,0.00033,0.00000,0.00000,0.00000


In [77]:
drop_list=drift_feature(X_dev, X_val, thresh=0.7)

These are no drifting features !


In [78]:
dev_final_iv, dev_IV = data_vars(df_dev, df_dev.badflag)
val_final_iv, val_IV = data_vars(df_val, df_val.badflag)
comp_iv=pd.merge(dev_IV, val_IV, on='VAR_NAME')
comp_iv = comp_iv.rename(columns={'IV_x': 'IV_dev', 'IV_y': 'IV_val'})
pd.options.display.float_format = '{:,.5f}'.format
comp_iv.IV_dev = comp_iv.IV_dev.round(4)
comp_iv.IV_val = comp_iv.IV_val.round(4)
comp_iv['Delta']=comp_iv['IV_val']-comp_iv['IV_dev']
comp_iv.sort_values(by='Delta', ascending=True)

,VAR_NAME,IV_dev,IV_val,Delta
6,ADA_3268,0.31920,0.26850,-0.05070
3,ADA_3208,0.21230,0.17580,-0.03650
9,ADA_3370,0.17310,0.14230,-0.03080
7,ADA_3273,0.22960,0.20690,-0.02270
13,ADA_3415,0.14620,0.12540,-0.02080
8,ADA_3348,0.10270,0.08320,-0.01950
36,__of_days_until_pmt_date,0.02400,0.00920,-0.01480
27,Fixed_Income,0.03130,0.01700,-0.01430
22,ADA_3926,0.05220,0.04210,-0.01010
11,ADA_3379,0.11130,0.10130,-0.01000


# char analysis for slownoappl segment

In [79]:
df_dev=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/noappl.csv')

df_val=pd.read_csv(s3_source_path+'/noappl.csv')

In [80]:
df_dev=df_dev[slownoappl_columns]
df_val=df_val[slownoappl_columns]

In [81]:
df_dev.fillna(0,inplace=True)
df_val.fillna(0,inplace=True)

X_dev=df_dev.drop(['badflag'],axis=1)
X_val=df_val.drop(['badflag'],axis=1)

In [82]:
ks_two_sample_test(X_dev, X_val)

,MOBVintageThread,__of_days_until_pmt_date,Times29_6mo,Times29_12mo,Times30_12mo,Times90_12mo,Times29_Ever,Times30_Ever,ADA_3863,ADA_3330,...,ADA_3012,ADA_3013,ADA_3167,ADA_3276,ADA_3335,ADA_3208,ADA_3386,ADA_3981,ADA_3987,Auto_Pay_flag
0,0.13120,0.01800,0.00299,0.01513,0.00615,0.00210,0.01153,0.00628,0.01927,0.04305,...,0.01528,0.01271,0.01487,0.01361,0.03009,0.04713,0.00176,0.03467,0.02131,0.00648
1,0.00000,0.00000,0.52042,0.00000,0.00737,0.89872,0.00000,0.00583,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.97617,0.00000,0.00000,0.00400


In [85]:
drop_list=drift_feature(X_dev, X_val, thresh=0.7)

These are no drifting features !


In [86]:
dev_final_iv, dev_IV = data_vars(df_dev, df_dev.badflag)
val_final_iv, val_IV = data_vars(df_val, df_val.badflag)
comp_iv=pd.merge(dev_IV, val_IV, on='VAR_NAME')
comp_iv = comp_iv.rename(columns={'IV_x': 'IV_dev', 'IV_y': 'IV_val'})
pd.options.display.float_format = '{:,.5f}'.format
comp_iv.IV_dev = comp_iv.IV_dev.round(4)
comp_iv.IV_val = comp_iv.IV_val.round(4)
comp_iv['Delta']=comp_iv['IV_val']-comp_iv['IV_dev']
comp_iv.sort_values(by='Delta', ascending=True)

,VAR_NAME,IV_dev,IV_val,Delta
8,ADA_3268,0.27380,0.16760,-0.10620
7,ADA_3208,0.10800,0.06010,-0.04790
20,ADA_3919,0.06710,0.03910,-0.02800
32,Times29_Ever,0.71570,0.68870,-0.02700
9,ADA_3276,0.06640,0.04000,-0.02640
15,ADA_3473,0.08850,0.06310,-0.02540
12,ADA_3379,0.09380,0.07230,-0.02150
11,ADA_3335,0.04150,0.02520,-0.01630
10,ADA_3330,0.07460,0.06010,-0.01450
17,ADA_3625,0.02810,0.01510,-0.01300


# char analysis for POT30 segment

In [87]:
df1=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202008/pot30.csv')
df2=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/pot30.csv')
df3=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202010/pot30.csv')
df_dev=pd.concat([df1,df2, df3],ignore_index=True)
df_val=pd.read_csv(s3_source_path+'/pot30.csv')

In [88]:
df_dev=df_dev[pot30_columns]
df_val=df_val[pot30_columns]

In [89]:
df_dev.fillna(0,inplace=True)
df_val.fillna(0,inplace=True)

X_dev=df_dev.drop(['badflag'],axis=1)
X_val=df_val.drop(['badflag'],axis=1)

In [90]:
ks_two_sample_test(X_dev, X_val)

,Loan_Type,Original_PI_Constant,number_of_prior_loans,MOBVintageUnthread,MOBVintageThread,yrs_on_job_dbl,__of_days_until_pmt_date,FICO_migration,NumberOfCars,MaxCarValue,...,ADA_3540,ADA_3879,ADA_3979,ADA_3144,Homeowner,FPD_n,PB,Coborrower,acflnps_n,state_risk
0,0.00898,0.01398,0.00643,0.04444,0.05745,0.00659,0.03215,0.03243,0.00558,0.00430,...,0.01768,0.00383,0.03958,0.01713,0.00060,0.00036,0.00719,0.00455,0.00437,0.00891
1,0.00502,0.00000,0.09313,0.00000,0.00000,0.07991,0.00000,0.00000,0.19851,0.49881,...,0.00000,0.64746,0.00000,0.00000,1.00000,1.00000,0.04330,0.42600,0.47871,0.00552


In [91]:
drop_list=drift_feature(X_dev, X_val, thresh=0.7)

These are no drifting features !


In [92]:
dev_final_iv, dev_IV = data_vars(df_dev, df_dev.badflag)
val_final_iv, val_IV = data_vars(df_val, df_val.badflag)
comp_iv=pd.merge(dev_IV, val_IV, on='VAR_NAME')
comp_iv = comp_iv.rename(columns={'IV_x': 'IV_dev', 'IV_y': 'IV_val'})
pd.options.display.float_format = '{:,.5f}'.format
comp_iv.IV_dev = comp_iv.IV_dev.round(4)
comp_iv.IV_val = comp_iv.IV_val.round(4)
comp_iv['Delta']=comp_iv['IV_val']-comp_iv['IV_dev']
comp_iv.sort_values(by='Delta', ascending=True)

,VAR_NAME,IV_dev,IV_val,Delta
13,ADA_3804,0.05450,0.00120,-0.05330
14,ADA_3805,0.04790,0.00130,-0.04660
37,__of_days_until_pmt_date,0.19680,0.16380,-0.03300
6,ADA_3368,0.07900,0.06360,-0.01540
31,Times29_12mo,0.14150,0.12740,-0.01410
29,Original_PI_Constant,0.03120,0.01830,-0.01290
16,ADA_3950,0.01670,0.00590,-0.01080
9,ADA_3474,0.08440,0.07600,-0.00840
3,ADA_3144,0.01030,0.00210,-0.00820
15,ADA_3879,0.03380,0.02610,-0.00770


# char analysis for pot60 segment

In [93]:
df1=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202008/pot60.csv')
df2=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/pot60.csv')
df3=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202010/pot60.csv')
df_dev=pd.concat([df1,df2, df3],ignore_index=True)
df_val=pd.read_csv(s3_source_path+'/pot60.csv')

In [94]:
df_dev=df_dev[pot60_columns]
df_val=df_val[pot60_columns]

In [95]:
df_dev.fillna(0,inplace=True)
df_val.fillna(0,inplace=True)

X_dev=df_dev.drop(['badflag'],axis=1)
X_val=df_val.drop(['badflag'],axis=1)

In [96]:
X_dev.shape

(22318, 37)

In [97]:
ks_two_sample_test(X_dev, X_val)

,number_of_prior_loans,MOBVintageThread,FICO_migration,NumberOfCars,MaxCarValue,Times29_6mo,Times30_6mo,Times60_9mo,Times29_Ever,Times60_Ever,...,ADA_3926,ADA_3195,ADA_3931,ADA_3374,ADA_3386,ADA_3762,ADA_3869,ADA_3144,ADA_3850,state_risk
0,0.01447,0.07507,0.03553,0.00671,0.00612,0.00392,0.00338,0.02146,0.02178,0.02169,...,0.04608,0.01906,0.02210,0.03196,0.01347,0.11665,0.01511,0.01733,0.01862,0.01717
1,0.17299,0.00000,0.00000,0.95492,0.98111,0.99999,1.00000,0.00925,0.00786,0.00822,...,0.00000,0.02878,0.00666,0.00001,0.23981,0.00000,0.13908,0.05993,0.03489,0.06391


In [98]:
drop_list=drift_feature(X_dev, X_val, thresh=0.7)


These are no drifting features !


In [99]:
dev_final_iv, dev_IV = data_vars(df_dev, df_dev.badflag)
val_final_iv, val_IV = data_vars(df_val, df_val.badflag)
comp_iv=pd.merge(dev_IV, val_IV, on='VAR_NAME')
comp_iv = comp_iv.rename(columns={'IV_x': 'IV_dev', 'IV_y': 'IV_val'})
pd.options.display.float_format = '{:,.5f}'.format
comp_iv.IV_dev = comp_iv.IV_dev.round(4)
comp_iv.IV_val = comp_iv.IV_val.round(4)
comp_iv['Delta']=comp_iv['IV_val']-comp_iv['IV_dev']
comp_iv.sort_values(by='Delta', ascending=True)

,VAR_NAME,IV_dev,IV_val,Delta
14,ADA_3804,0.08550,0.00000,-0.08550
13,ADA_3762,0.04060,0.01730,-0.02330
12,ADA_3759,0.07770,0.05950,-0.01820
26,FICO_migration,0.04070,0.03280,-0.00790
6,ADA_3386,0.05580,0.04930,-0.00650
28,MaxCarValue,0.03940,0.03300,-0.00640
18,ADA_3869,0.02210,0.01650,-0.00560
2,ADA_3195,0.01500,0.00970,-0.00530
0,ADA_3117,0.01500,0.00980,-0.00520
19,ADA_3880,0.00530,0.00020,-0.00510


# char analysis for pot90 segment

In [40]:
df1=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202008/pot90.csv')
df2=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202009/pot90.csv')
df3=pd.read_csv('s3://mf-ml-ds/propensity_to_pay/Monthly_Validation/val_202010/pot90.csv')
df_dev=pd.concat([df1,df2, df3],ignore_index=True)
df_val=pd.read_csv(s3_source_path+'/pot90.csv')

In [41]:
df_dev=df_dev[pot90_columns]
df_val=df_val[pot90_columns]

In [42]:
df_dev.fillna(0,inplace=True)
df_val.fillna(0,inplace=True)

X_dev=df_dev.drop(['badflag'],axis=1)
X_val=df_val.drop(['badflag'],axis=1)

In [43]:
ks_two_sample_test(X_dev, X_val)

,MOBVintageThread,yrs_on_job_dbl,number_of_payments,NumberOfCars,MaxCarYear,Times29_6mo,Times30_6mo,Times90_6mo,Times29_Ever,Times30_Ever,...,ADA_3410,ADA_3540,ADA_3761,ADA_3827,ADA_3012,ADA_3013,ADA_3914,ADA_3963,upb_ratio,state_risk
0,0.04580,0.03131,0.04076,0.00565,0.00279,0.04080,0.00688,0.02931,0.01162,0.01410,...,0.02781,0.03646,0.20241,0.02642,0.08134,0.08359,0.05174,0.06179,0.02489,0.00955
1,0.00000,0.00000,0.00000,0.77310,0.99993,0.00000,0.53460,0.00000,0.04931,0.00855,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.16363


In [44]:
drop_list=drift_feature(X_dev, X_val, thresh=0.7)


These are no drifting features !


In [45]:
dev_final_iv, dev_IV = data_vars(df_dev, df_dev.badflag)
val_final_iv, val_IV = data_vars(df_val, df_val.badflag)
comp_iv=pd.merge(dev_IV, val_IV, on='VAR_NAME')
comp_iv = comp_iv.rename(columns={'IV_x': 'IV_dev', 'IV_y': 'IV_val'})
pd.options.display.float_format = '{:,.5f}'.format
comp_iv.IV_dev = comp_iv.IV_dev.round(4)
comp_iv.IV_val = comp_iv.IV_val.round(4)
comp_iv['Delta']=comp_iv['IV_val']-comp_iv['IV_dev']
comp_iv.sort_values(by='Delta', ascending=True)

,VAR_NAME,IV_dev,IV_val,Delta
13,ADA_3759,0.21050,0.07100,-0.13950
17,ADA_3827,0.10150,0.03780,-0.06370
32,Times90_6mo,0.17310,0.13620,-0.03690
33,number_of_payments,0.16980,0.14060,-0.02920
16,ADA_3804,0.02380,0.00080,-0.02300
24,MOBVintageThread,0.15010,0.12850,-0.02160
31,Times60_Ever,0.20470,0.18660,-0.01810
36,yrs_on_job_dbl,0.02300,0.00600,-0.01700
2,ADA_3164,0.03070,0.01400,-0.01670
35,upb_ratio,0.07960,0.06330,-0.01630
